In [ ]:
import os
import shutil
import sys
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import numpy as np
import pandas as pd
from bokeh.plotting import figure
from bokeh.plotting import show
from bokeh.plotting import output_notebook
from bokeh.layouts import column, row, layout, gridplot
from bokeh.models import Spacer
from bokeh.models import HoverTool, ColumnDataSource
from dateutil import tz
JST = tz.gettz('Asia/Tokyo')
UTC = tz.gettz('utc')

from common import Indicators, Columns, Signal
from technical import SQUEEZER
from time_utils import TimeFilter, TimeUtils
from backtest_squeezer import Backtest
from time_chart import TimeChart
output_notebook()


    


In [ ]:
def plot_profit(chart, df, t0, t1, cl):
    df['texit'] = pd.to_datetime(df['exit_time'])
    df['tentry'] = pd.to_datetime(df['entry_time'])
    df2 = df[df['texit'] >= t0]
    df2 = df2[df2['texit'] <= t1]
    print('trade count: ', len(df2))
    
    no = df2['no'].to_list()
    sls = df2['sl'].to_list()
    signal = df2['signal'].to_list()
    tentry = df2['tentry'].to_list()
    price1 = df2['entry_price'].to_list()
    texit = df2['texit'].to_list()
    price2 = df2['exit_price'].to_list()
    profits = df2['profit'].to_list()
    for n, sig, ten, tex, p1, p2, sl, prof in zip(no, signal, tentry, texit, price1, price2, sls, profits):
        if sig == 1:
            color='green'
        elif sig == -1:
            color='red'
        else:
            continue
        chart.vline(ten, color=color)
        chart.vline(tex, color='gray')
        color = 'blue' if prof > 0 else 'red'
        vmax = np.nanmax(cl)
        vmin = np.nanmin(cl)
        h = (vmax - vmin) * 0.1
        chart.text(tex, p2 - h * (n % 3), f'{prof:.3f}', color=color)
            



def main():
    symbol = 'NSDQ'
    timeframe = 'M15'
    days = 10

    dirpath = f'./debug/squeezer/{symbol}/{timeframe}'
    os.makedirs(dirpath, exist_ok=True)
    test = Backtest(symbol, timeframe)
    df = test.evaluate(dirpath)
    time = test.data[Columns.JST]
    t0 = time[0]
    t1 = t0 + timedelta(days=days)

    count = 0
    while t1 < time[-1]:
        n, data = TimeUtils.slice(test.data, Columns.JST, t0, t1)   
        if n < 10:
            t0 = t1
            t1 = t0 + timedelta(days=days)
            continue
        
        jst = data[Columns.JST]
        cl = data[Columns.CLOSE]

        sqz = data[Indicators.SQUEEZER]
        std = data[Indicators.SQUEEZER_STD]
        atr = data[Indicators.SQUEEZER_ATR]
        upper = data[Indicators.SQUEEZER_UPPER]
        lower = data[Indicators.SQUEEZER_LOWER]
        signal = data[Indicators.SQUEEZER_SIGNAL]
        entry = data[Indicators.SQUEEZER_ENTRY]
        ext = data[Indicators.SQUEEZER_EXIT]

        chart1 = TimeChart('', 800, 300, jst)
        chart1.line(cl, color='blue', alpha=0.4)
        chart1.line(upper, color='green', alpha=0.4)
        chart1.line(lower, color='red', alpha=0.4)
        chart1.markers(signal, cl, 1, marker='o', color='rgba(0, 255, 0, 0.2)', size=10)
        chart1.markers(signal, cl, -1, marker='o', color='rgba(255, 0, 0, 0.2)', size=10)
        chart1.markers(entry, cl, 1, marker='^', color='rgba(0, 255, 0, 0.2)', size=10)
        chart1.markers(entry, cl, -1, marker='v', color='rgba(255, 0, 0, 0.2)', size=10)
        plot_profit(chart1, df, t0, t1, cl)
        
        chart2 = TimeChart('', 800, 300, jst)
        chart2.line(std, color='blue', alpha=0.4)
        chart2.line(atr, color='red', alpha=0.4)
        
        figs = [chart1.fig, chart2.fig]
        l = column(*figs, background='gray')
        show(l)
        
        
        count += 1 
        t0 = t1
        t1 = t0 + timedelta(days=days)
        
        
if __name__ == '__main__':
    main()
        
